In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14510,2024-03-07,Eua Ncaa,21:30,Stephen F. Austin,Southern Utah,1.36,3.33,148.5,1.80,1.95,0.0,0.00,0.00,0ppGfPWq,0.735294,0.300300,0.555556,0.512821,0.035594,186.482,133.564484,0.716233,0.6,1.341641,2.236068,417.60,3.242,2.344178,0.723065,-50.0,196.340,104.109293,0.530250,0.0,0.000000,NaN,180.94,2.528,1.098121,0.434383,-32.0,58,83,7.20,2.18,101.042,260.210,0.594030,0.056569,NaN,-2.60,-0.520,-0.692308,0.748061,0.6,-0.148061,1.37,0.274,8.503650,0.303126,0.2,-0.103126
14511,2024-03-07,Eua Ncaa,21:00,Illinois State,Evansville,1.65,2.33,135.5,1.80,1.95,-4.5,2.00,3.30,6BKPO7lt,0.606061,0.429185,0.555556,0.512821,0.035245,170.520,95.228452,0.558459,1.8,1.643168,0.912871,104.40,2.808,2.628330,0.936015,10.0,250.876,94.641675,0.377245,0.0,0.000000,NaN,339.90,3.674,1.331946,0.362533,-57.0,72,66,1.45,5.15,138.380,296.770,0.241624,0.056569,0.346883,0.09,0.018,36.111111,0.578071,0.5,-0.078071,-3.00,-0.600,-2.216667,0.232880,0.2,-0.032880
14512,2024-03-07,Eua Ncaa,21:30,N.Carolina Central,South Carolina St,1.31,3.63,143.5,1.91,1.91,0.0,0.00,0.00,jqxIg4Pi,0.763359,0.275482,0.523560,0.523560,0.038841,134.688,37.663870,0.279638,1.8,1.643168,0.912871,122.45,1.640,0.565951,0.345092,55.0,135.246,73.178141,0.541074,3.0,0.000000,0.000000,81.74,1.934,0.889146,0.459744,36.0,79,61,1.55,1.34,108.186,175.664,0.664165,0.000000,NaN,0.87,0.174,1.781609,0.000000,0.0,0.000000,3.61,0.722,3.642659,0.335156,0.5,0.164844
14513,2024-03-07,Eua Ncaa,21:30,Morgan State,Coppin State,1.31,3.63,140.5,1.80,1.95,0.0,0.00,0.00,A5h1SRQh,0.763359,0.275482,0.555556,0.512821,0.038841,210.868,80.307812,0.380844,0.6,1.341641,2.236068,217.36,2.918,1.026387,0.351743,-33.0,349.910,220.231659,0.629395,0.0,0.000000,NaN,187.34,5.370,3.009227,0.560378,-47.0,88,58,2.47,3.23,373.036,411.150,0.664165,0.056569,NaN,0.55,0.110,2.818182,0.000000,0.0,0.000000,-5.00,-1.000,-2.630000,0.145003,0.0,-0.145003
14514,2024-03-07,Eua Ncaa,21:00,Youngstown State,Cleveland State,1.23,4.44,151.5,1.91,1.91,0.0,0.00,0.00,U5qOCgi9,0.813008,0.225225,0.523560,0.523560,0.038233,108.048,26.316200,0.243560,2.4,1.341641,0.559017,71.07,1.406,0.307945,0.219022,32.0,106.762,35.056097,0.328357,1.8,1.643168,0.912871,89.25,1.466,0.487986,0.332869,20.0,69,85,1.03,1.05,100.774,145.074,0.800639,0.000000,NaN,-0.21,-0.042,-5.476190,0.784698,0.8,0.015302,0.27,0.054,63.703704,0.504092,0.5,-0.004092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14623,2024-03-07,China Wcba Feminina,08:30,Sichuan F,Guangdong F,1.13,5.35,155.5,1.82,1.88,0.0,0.00,0.00,tlTWITdU,0.884956,0.186916,0.549451,0.531915,0.071872,514.532,910.059788,1.768714,3.0,0.000000,0.000000,2142.00,5.158,8.860066,1.717733,144.0,490.002,864.905231,1.765106,3.0,0.000000,0.000000,92.92,5.114,8.882231,1.736846,160.0,102,92,21.00,1.01,99.170,528.636,0.920985,0.022933,NaN,0.19,0.038,3.421053,0.000000,0.0,0.000000,19.74,3.948,1.101824,0.000000,0.0,0.000000
14624,2024-03-07,China Wcba Feminina,08:30,Shaanxi Tianze F,Fujian F,2.05,1.70,141.5,1.85,1.85,1.5,1.88,1.58,I94xvDSu,0.487805,0.588235,0.540541,0.540541,0.076040,545.496,513.334720,0.941042,0.6,1.341641,2.236068,4

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14511,21:00,Eua Ncaa,Illinois State,Evansville,1.65,Back Home
14515,21:00,Eua Ncaa,Temple,UAB,2.14,Back Home
14525,21:00,Eua Ncaa,Austin Peay,North Alabama,1.58,Back Home
14545,16:15,Islândia Premier League,Thorl,Njardvik,1.70,Back Home
14559,22:30,Eua Ncaa,Georgia State,Marshall,1.83,Back Home
14613,14:00,Europa Eurocopa Feminina,Besiktas F,Uni Girona F,1.69,Back Home
14617,00:00,Eua Ncaa,Siu Edwardsville,Eastern Illinois,1.67,Back Home
